In [1]:
from typing import List, Tuple
from datetime import datetime
import pandas as pd
import json
import os

In [2]:
#Lectura de base de datos con pandas

file_path = "farmers-protest-tweets-2021-2-4.json"
%load_ext memory_profiler
%load_ext line_profiler

A partir de la primera version se puede notar claramente que el cuello de botella corresponde a la lectura y parseo del archivo Json, en donde incluso se observa que el alrededor del 92% del tiempo de uso llega a ser por el tiempo de lectura del archivo json

Para la lectura del archivo Json se creará en este caso una función que optimice el tiempo de lectura del archivo json y retornado solo los campos deseados

1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene
por cada uno de esos días. Debe incluir las siguientes funciones:

In [77]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Lectura del archivo y obteniendo los datos que necesitamos
    file1 = open(file_path, 'r')
    Lines = file1.readlines()
    data = []

    for line in Lines:
        json_value = json.loads(line)
        user = json_value.get("user").get("username")
        date = datetime.strptime(json_value.get("date")[:10], "%Y-%m-%d").date()
        data_id = json_value.get("id")
        data.append({"date":date, "user":user, "id":data_id})
    
    data = pd.DataFrame(data)
    
    #top 10 days
    top_10_days = tweets_data.groupby(["date"]).count()
    top_10_days = top_10_days.sort_values("id", ascending=False).head(10)
    top_10_days = list(top_10_days.index)
    
    #filter data and group by date and user
    tweets_data = tweets_data.loc[tweets_data["date"].isin(top_10_days)]
    tweets_data = tweets_data.groupby(["date","user"]).count()

    tweets_data = tweets_data.sort_values(["date","id"], ascending=False)
    tweets_data = tweets_data.reset_index().groupby("date").first()
    
    tweets_data = [tuple(i) for i in tweets_data[["user"]].itertuples()]
    return tweets_data

In [83]:
%timeit q1_time(file_path)

11.3 s ± 1.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Como se puede ver en este caso el valor de lectura de tiempo se reduce considerablemente

Para el caso de optimizacion de memoria utilizaremos archivos en disco que almacenen datos de manera temporal 

Se dividirá la lectura en varios lotes, aplicando una tecnica de map reduce y guardando los archivos con los resultados previos

In [42]:
def q1_memory(file_path):
    # Create dict of date values and num of lines in file
    file_lines = {}

    # Open the file for reading
    with open(file_path, 'r') as file:
        for line in file:
            json_value = json.loads(line)
            date = json_value.get("date")[:10]
            user = json_value.get("user").get("username")
            data_id = json_value.get("id")
            data = {"user":user, "id":data_id}
            with open(f"data_q1/{date}","a") as fwrite:
                fwrite.write(json.dumps(data)+"\n")
                try:
                    file_lines[date] += 1 
                except:
                    file_lines[date] = 0
    
    top_10_days = pd.DataFrame([{"date":i, "rows":file_lines[i]} for i in file_lines])
    top_10_days = top_10_days.sort_values("rows", ascending = False)[:10]
    top_10_days = list(top_10_days['date'])
    
    result_list = []
    for i in top_10_days:
        data_tmp = pd.read_json(f'data_q1/{i}', orient='records', lines=True)
        data_tmp = data_tmp.groupby("user").count().sort_values("id", ascending = False)[:1]
        user = data_tmp.index.values[0]
        result_list.append((i, user))
    
    # delete all previous files generated
    directory_path = 'data_q1'
    file_list = os.listdir(directory_path)
    for file_name in file_list:
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)

    return result_list


In [44]:
%memit q1_memory(file_path)

peak memory: 149.06 MiB, increment: 10.73 MiB


2. Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:

In [7]:
import emoji

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    #En este caso haremos la lectura igual que en la primera pregunta pero extraeremos unicamente el campo content
    file_data = open(file_path, 'r')
    Lines = file_data.readlines()
    data = []

    for line in Lines:
        json_value = json.loads(line)
        data.append(json_value.get("content"))

    emoji_values = []
    
    for i in data:
        emoji_values += [value.chars for value in emoji.analyze(i)]

    #Vamos a crear un dataframe con los resultados
    data = pd.DataFrame({"emoji":emoji_values})
    data["counter"] = 1
    data = data.groupby('emoji').sum().sort_values("counter", ascending = False).head(10)
    emoji_list = [tuple(i) for i in data[["counter"]].itertuples()]
    file_data.close() #Liberamos memoria
    return emoji_list

In [8]:
%memit resultado_q2_time = q2_time(file_path)

peak memory: 594.25 MiB, increment: 20.25 MiB


In [113]:
%timeit resultado_q2_time = q2_time(file_path)

30.6 s ± 2.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [112]:
# Imprimir resultado
resultado_q2_time

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

In [28]:
import emoji

def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    # Para evitar consumir la memoria en la lectura del archivo iremos leyendo linea a linea el archivo
    # haremos una cuenta un agrupamiento previo e iremos almacenando en un archivo

    # Lectura linea a linea
    file1 = open(file_path,"r")
    fwrite = open("aux_mem_q2/emoji_data","a")
    file_read = open("aux_mem_q2/emoji_data","r")

    for line in file1:
        json_value = json.loads(line)
        # Leeremos solo el valor del content donde estar los emojis
        content = json_value.get("content")
        
        #Extraemos emojis y los agrupamos
        tmp_emoji_list = [value.chars for value in emoji.analyze(content)]
        tmp_emoji_list = '\n'.join(tmp_emoji_list)
        
        if tmp_emoji_list: #Emojis existen
            fwrite.write(tmp_emoji_list+"\n")

    file1.close()
    fwrite.close()
        
    # Ahora leeremos el archivo fila a fila y almacenaremos en un dictionario de datos sumando uno por cada ocurrencia
    emoji_values = {}

    for emoji_line in file_read:
        if emoji_line.replace('\n','') in emoji_values.keys():
            emoji_values[emoji_line.replace('\n','')] +=1 # Si existe se suma uno
        else:
            emoji_values[emoji_line.replace('\n','')] = 0 # Si no existe lo crea
    
    #Lo hacemos dataframe para agrupar y sacar los maximos
    emoji_values = pd.DataFrame({"emoji":list(emoji_values.keys()),"conteo":list(emoji_values.values())})
    emoji_values = emoji_values.sort_values('conteo', ascending=False).head(10)

    # #Lo volvemos listado de tuplas
    emoji_values = [tuple(i) for i in emoji_values.set_index('emoji').itertuples()]

    # Borrar el archivo generado
    os.remove('aux_mem_q2/emoji_data')
    file1.close()
    
    return emoji_values

In [29]:
%memit x = q2_memory(file_path)

peak memory: 158.62 MiB, increment: 0.00 MiB


In [30]:
x

[('🙏', 15146),
 ('😂', 9215),
 ('🚜', 8915),
 ('🌾', 6545),
 ('🇮🇳', 6257),
 ('🤣', 5003),
 ('✊', 4952),
 ('❤️', 4145),
 ('🙏🏻', 3950),
 ('💚', 3119)]

3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@)
que registra cada uno de ellos. Debe incluir las siguientes funciones:

In [44]:
def q3_time(file_path: str) -> List[Tuple[str, int]]:
    # De igual manera se hara una lectura del archivo linea a linea para tomar los campos estrictamente necesarios
    # La documentacion no lo especifica pero en un analsis del archivo se puede observar que existe el campo mentionedUsers del cual se tomara los usernames
    file1 = open(file_path, 'r')
    Lines = file1.readlines()
    list_mentioned_user = []
    
    for line in Lines:
        json_value = json.loads(line)
        mentioned_users = json_value.get("mentionedUsers")
        if mentioned_users:
            list_mentioned_user += [i["username"] for i in mentioned_users] #Se tomara las n veces que se mecione en el tweet, se puede tomar unicos transformando a set y luego a list, pero en este caso lo mantedre así
    
    # Transformamos a dataframe para ordenar ocurrencias
    mentioned_users = pd.DataFrame({"mentioned_user":list_mentioned_user})
    mentioned_users["conteo"] = 1
    mentioned_users = mentioned_users.groupby("mentioned_user").sum()
    mentioned_users = mentioned_users.sort_values("conteo", ascending = False).head(10)

    #Transformamos a tuplas
    mentioned_users = [tuple(i) for i in mentioned_users.itertuples()]
    
    return mentioned_users

In [46]:
%timeit q3_time(file_path)

6.36 s ± 590 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%memit q3_time(file_path)

peak memory: 582.40 MiB, increment: 412.10 MiB


In [54]:
q3_time(file_path)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

In [3]:
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    # De igual manera se hara una lectura del archivo linea a linea para tomar los campos estrictamente necesarios
    # Se toma el mismo campo mentionedUsers del cual se tomara los usernames
    file1 = open(file_path, 'r')
    list_mentioned_user = []
    
    #La lectura es linea a linea para no saturar memoria
    for line in file1:
        json_value = json.loads(line)
        mentioned_users = json_value.get("mentionedUsers")
        if mentioned_users:
            list_mentioned_user += [i["username"] for i in mentioned_users] #Se tomara las n veces que se mecione en el tweet, se puede tomar unicos transformando a set y luego a list, pero en este caso lo mantedre así
    
    # Transformamos a dataframe para ordenar ocurrencias
    mentioned_users = pd.DataFrame({"mentioned_user":list_mentioned_user})
    mentioned_users["conteo"] = 1
    mentioned_users = mentioned_users.groupby("mentioned_user").sum()
    mentioned_users = mentioned_users.sort_values("conteo", ascending = False).head(10)

    #Transformamos a tuplas
    mentioned_users = [tuple(i) for i in mentioned_users.itertuples()]

    file1.close()
    return mentioned_users

In [50]:
%timeit q3_memory(file_path)

11.1 s ± 960 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%memit q3_memory(file_path)

peak memory: 150.88 MiB, increment: 6.93 MiB


In [6]:
q3_memory(file_path)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]